In [138]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc

In [8]:
bpl.output_notebook()

Loading BokehJS ...

In [9]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "05/18/2020"

In [10]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Adquisición de datos históricos, esta parte solo es necesaria una vez, guarda los archivos a disco

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [11]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [12]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [13]:
len(frames)

37

In [14]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [15]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [16]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [26]:
confirmados = all_data[all_data["RESULTADO"]==1]

In [79]:
por_fecha = confirmados.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [120]:
fechas_validas = por_fecha.columns

In [139]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [91]:
conf_acum = por_fecha.sum().to_frame("Confirmados").reset_index()

In [182]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [183]:
p.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=1,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='16475', ...)

In [184]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,10)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [185]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [186]:
bpl.show(p)